# Country_level

## First Part

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.common.by import By
# Change the driver path to your computer
driver_path = r'C:\Users\panda\Downloads\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome(driver_path)
#Open the website
driver.get("https://climexp.knmi.nl/plot_atlas_form.py")
#1. Select "countries"
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[2]/td[2]/input[7]").click()
#2. Select "time series"
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div/div[1]/div[2]/div/form/table/tbody/tr[10]/td[2]/input[3]").click()
#3. Check RCPs
#rcp2.6
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[12]/td[2]/input[1]").click()
#rcp4.5 has been clicked by default
#rcp6.0
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[12]/td[2]/input[3]").click()
#rcp8.5
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[12]/td[2]/input[4]").click()

#4. input years for each country
## Clear values first
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div/div[1]/div[2]/div/form/table/tbody/tr[13]/td[2]/input[1]").clear()
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[13]/td[2]/input[2]").clear()
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div/div[1]/div[2]/div/form/table/tbody/tr[13]/td[2]/input[1]").send_keys(2005)
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[13]/td[2]/input[2]").send_keys(2005)

#4.5 Get rid of some useless countries
optionsList = pd.read_excel('optionsList.xlsx')
optionsList = list(optionsList['Country'])

#5. Looping the countries
for optionValue in optionsList:
    print (f"starting loop on option: {optionValue}")
    select = Select(driver.find_element(by=By.XPATH, value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[3]/td[2]/select"))
    select.select_by_value(optionValue)

    #6. Click "Make time series"
    driver.find_element(by=By.XPATH, 
                        value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[16]/td/input").click()
        
    #This step will take 15 minutes to 1 hour to extracte data files
    #since they used some kind of .py file.
    try:
        element = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((by=By.XPATH, 
                value = "/html/body/div[3]/div[2]/a[5]"))
        )
    except:
        element = WebDriverWait(driver, 3600).until(
            EC.presence_of_element_located((by=By.XPATH, 
                value = "/html/body/div[3]/div[2]/a[5]"))
        )
        
    #7. Get all data
    driver.find_element(by=By.XPATH, 
                value = "/html/body/div[3]/div[2]/a[5]").click()
    try:
        element = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((by=By.XPATH, 
                value = "/html/body/div[3]/div[2]/a[5]"))
        )
    except:
        element = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((by=By.XPATH, 
                value = "/html/body/div[3]/div[2]/a[5]"))
        )
    #8. Download data
    driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div/div[1]/p/a").click()
    if optionValue == 'Zimbabwe':
        driver.quit()
    else:
        driver.back()

## Second Part

In [ ]:
import os
import zipfile
import pandas as pd
## Get all zipped file names
filelist = []
# Please confirm the download location
path = "C:/Users/panda/Downloads/Country/"
for file in os.listdir(f"{path}"):
    if file.endswith(".zip"):
        filelist.append(file)
## First loop for extracting all files
for i in range(len(filelist)):
    filepath = path + filelist[i]
    unzip=zipfile.ZipFile(filepath)
    unzip.extractall(f'{path}')
    unzip.close()
    rcppath = path + 'atlas/series/CMIP5one'
    # list rcp folders 
    rcp = [dI for dI in os.listdir(f'{rcppath}') if os.path.isdir(os.path.join(f'{rcppath}',dI))]
    ### First.1 loop for getting rcp folders path & get all countries for each rcp
    for j in rcp[1:]:
        RCP = rcppath + '/' + j 
        monthly = [dI for dI in os.listdir(f'{RCP}') if os.path.isdir(os.path.join(f'{RCP}',dI))]
        # 1850-2005
        MON1 = RCP + '/' + monthly[0]
        country = [dI for dI in os.listdir(f'{MON1}') if os.path.isdir(os.path.join(f'{MON1}',dI))]
## Second loop for read rcp files then get the paths of hist (1850 - 2005) and future data (2006 - 2100)
for j in rcp[1:]:
    RCP = rcppath + '/' + j 
    monthly = [dI for dI in os.listdir(f'{RCP}') if os.path.isdir(os.path.join(f'{RCP}',dI))]
    # 1850-2005
    MON1 = RCP + '/' + monthly[0]
    # 2006-2100
    MON2 = RCP + '/' + monthly[1]
    ### Second.1 loop to get all hist and future data
    for q in range(len(country)):
        # 1850-2005
        CON1 = MON1 + '/' + country[q]
        txtlist1 = []
        for txt1 in os.listdir(f"{CON1}"):
            if txt1.endswith(".txt"):
                txtlist1.append(txt1)
        # 2006-2100
        CON2 = MON2 + '/' + country[q]
        txtlist2 = []
        for txt2 in os.listdir(f"{CON2}"):
            if txt2.endswith(".txt"):
                txtlist2.append(txt2)
                
                
        #### Second.1.1 loop to read all hist data and caculate the mean
        for k in range(len(txtlist1)):
            if k == 0:
                txtfile = CON1 + '/' + txtlist1[k]
                df = pd.read_csv(f"{txtfile}", on_bad_lines='skip')
                try: # some txt file doesnot contain data.
                    df = df[-156:]
                    df.rename(columns= {df.columns[0]:f'Model{k}'}, inplace = True)
                    df = df[f'Model{k}'].str.rsplit("       ",expand=True)
                    df = df[1].str.rsplit("     ", expand=True)
                    df = df[1].str.rsplit("   ", expand=True).add_prefix('nCol_').join(df)
                    df = df[[0,'nCol_0']]
                    df.rename(columns={0: f'Model{k}', 'nCol_0': 'Year'}, inplace=True)
                except:
                    continue
            else:
                txtfile = CON1 + '/' + txtlist1[k]
                dt = pd.read_csv(f"{txtfile}", on_bad_lines='skip')
                try: # some txt file doesnot contain data.
                    dt = dt[-156:]
                    dt.rename(columns= {dt.columns[0]:f'Model{k}'}, inplace = True)
                    dt = dt[f'Model{k}'].str.rsplit("       ",expand=True)
                    dt = dt[1].str.rsplit("     ", expand=True)
                    dt = dt[1].str.rsplit("   ", expand=True).add_prefix('nCol_').join(dt)
                    dt = dt[[0,'nCol_0']]
                    dt.rename(columns={0: f'Model{k}', 'nCol_0': 'Year'}, inplace=True)
                except:
                    continue

                df = df.merge(dt, on = 'Year', how = 'left')

        df = df.astype(float)
        df = df.set_index('Year')
        df['mean'] = df.mean(axis=1)
        country1 = [country[q]]*len(df)
        df['Country'] = country1
        df = df[['mean','Country']]
        df.rename(columns= {'mean':f'{j}'},inplace = True)
        dfhist = df 
        
        #### Second.1.2 loop to read all future data and caculate the mean
        df  = pd.DataFrame()
        for k in range(len(txtlist2)):
            if k == 0:
                txtfile = CON2 + '/' + txtlist2[k]
                df = pd.read_csv(f"{txtfile}", on_bad_lines='skip')
                try: # some txt file doesnot contain data.
                    df = df[-95:]
                    df.rename(columns= {df.columns[0]:f'Model{k}'}, inplace = True)
                    df = df[f'Model{k}'].str.rsplit("       ",expand=True)
                    df = df[1].str.rsplit("     ", expand=True)
                    df = df[1].str.rsplit("   ", expand=True).add_prefix('nCol_').join(df)
                    df = df[[0,'nCol_0']]
                    df.rename(columns={0: f'Model{k}', 'nCol_0': 'Year'}, inplace=True)
                except:
                    continue
                    
            else:
                txtfile = CON2 + '/' + txtlist2[k]
                dt = pd.read_csv(f"{txtfile}", on_bad_lines='skip')
                try: # some txt file doesnot contain data.
                    dt = dt[-95:]
                    dt.rename(columns= {dt.columns[0]:f'Model{k}'}, inplace = True)
                    dt = dt[f'Model{k}'].str.rsplit("       ",expand=True)
                    dt = dt[1].str.rsplit("     ", expand=True)
                    dt = dt[1].str.rsplit("   ", expand=True).add_prefix('nCol_').join(dt)
                    dt = dt[[0,'nCol_0']]
                    dt.rename(columns={0: f'Model{k}', 'nCol_0': 'Year'}, inplace=True)
                except:
                    continue

                df = df.merge(dt, on = 'Year', how = 'left')

        df = df.astype(float)
        df = df.set_index('Year')
        df['mean'] = df.mean(axis=1)
        country2 = [country[q]]*len(df)
        df['Country'] = country2
        df = df[['mean','Country']]
        df.rename(columns= {'mean':f'{j}'},inplace = True)
        dffuture = df

        ## Combine hist and future
        # Create new folders under current directory: country/rcp26,rcp45,rcp60,rcp85
        dd = pd.concat([dfhist, dffuture])
        dd.to_excel(f'country/{j}/{country[q]}{j}.xlsx')

## Third Part

In [ ]:
dire = 'C:/Users/panda/OneDrive - University of Denver/Pardee Center/17 2022 Summer/country'
# dire needs to change
subfolders = [ f.path for f in os.scandir(f"{dire}") if f.is_dir() ]
for p in subfolders:
    excellist = []
    for excel in os.listdir(f"{p}"):
        if excel.endswith(".xlsx"):
            excellist.append(excel)
        for a in range(len(excellist)):
            value = excellist[a][:-5]
            value = value[-5:]
            if a == 0:
                df = pd.read_excel(f'{p}/{excellist[a]}')
            else:
                dt = pd.read_excel(f'{p}/{excellist[a]}')

            final = pd.concat([df,dt])
            final = final.pivot_table(index = 'Country', columns = 'Year', values = f'{value}')
        
        final.to_excel(f"{p}/{value}_countrylevel.xlsx")

# Region_level

## First Part

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.common.by import By
# Change the driver path to your computer
driver_path = r'C:\Users\panda\Downloads\chromedriver_win32\chromedriver.exe'
driver = webdriver.Chrome(driver_path)
#Open the website
driver.get("https://climexp.knmi.nl/plot_atlas_form.py")
#1. Select "time series"
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div/div[1]/div[2]/div/form/table/tbody/tr[10]/td[2]/input[3]").click()
#2. Check RCPs
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[12]/td[2]/input[1]").click()
#rcp4.5 has been clicked by default
#driver.find_element(by=By.XPATH, 
                    #value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[12]/td[2]/input[2]").click()
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[12]/td[2]/input[3]").click()
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[12]/td[2]/input[4]").click()
#3. input years for each region
## Clear values first
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div/div[1]/div[2]/div/form/table/tbody/tr[13]/td[2]/input[1]").clear()
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[13]/td[2]/input[2]").clear()
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div/div[1]/div[2]/div/form/table/tbody/tr[13]/td[2]/input[1]").send_keys(2005)
driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[13]/td[2]/input[2]").send_keys(2005)

# Get the full list of regions
select = driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[3]/td[2]/select")  #get the select element            
options = select.find_elements(by=By.TAG_NAME, value="option") #get all the options into a list

optionsList = []

for option in options: #iterate over the options, place attribute value in list
    optionsList.append(option.get_attribute("value"))
    
#4. Looping the regions
for optionValue in optionsList:
    print (f"starting loop on option: {optionValue}")
    select = Select(driver.find_element(by=By.XPATH, value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[3]/td[2]/select"))
    select.select_by_value(optionValue)
    
    #5. Click "Make time series"
    driver.find_element(by=By.XPATH, 
                        value = "/html/body/div[3]/div[1]/div[1]/div[2]/div/form/table/tbody/tr[16]/td/input").click()
        
    #This step will take 15 minutes to 1 hour to extracte data files
    #since they used some kind of .py file.
    
    try:
        element = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((by=By.XPATH, 
                value = "/html/body/div[3]/div[2]/a[5]"))
        )
    except:
        element = WebDriverWait(driver, 3600).until(
            EC.presence_of_element_located((by=By.XPATH, 
                value = "/html/body/div[3]/div[2]/a[5]"))
        )
        
    #6. Get all data
    driver.find_element(by=By.XPATH, 
                value = "/html/body/div[3]/div[2]/a[5]").click()
    try:
        element = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((by=By.XPATH, 
                value = "/html/body/div[3]/div[2]/a[5]"))
        )
    except:
        element = WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((by=By.XPATH, 
                value = "/html/body/div[3]/div[2]/a[5]"))
        )
    #7. Download data
    driver.find_element(by=By.XPATH, 
                    value = "/html/body/div[3]/div/div[1]/p/a").click()
    if optionValue == 'Antarcticsea':
        driver.quit()
    else:
        driver.back()

## Second Part

In [14]:
import os
import zipfile
import pandas as pd

filelist = []
# Please confirm the download location
path = "C:/Users/panda/Downloads/Region/"
## Get all zipped file names
for file in os.listdir(f"{path}"):
    if file.endswith(".zip"):
        filelist.append(file)
## First loop for extracting all files
for i in range(len(filelist)):
    filepath = path + filelist[i]
    unzip=zipfile.ZipFile(filepath)
    unzip.extractall(f'{path}')
    unzip.close()
    rcppath = path + 'atlas/series/CMIP5one'
    ### First.1 loop for getting rcp folders path
    # list rcp folders 
    rcp = [dI for dI in os.listdir(f'{rcppath}') if os.path.isdir(os.path.join(f'{rcppath}',dI))]
    #get all regions for each rcp
    for j in rcp[1:]:
        RCP = rcppath + '/' + j 
        monthly = [dI for dI in os.listdir(f'{RCP}') if os.path.isdir(os.path.join(f'{RCP}',dI))]
        # 1850-2005
        MON1 = RCP + '/' + monthly[0]
        region = [dI for dI in os.listdir(f'{MON1}') if os.path.isdir(os.path.join(f'{MON1}',dI))]
## Second loop for read rcp files then get the paths of hist (1850 - 2005) and future data (2006 - 2100)
for j in rcp[1:]:
    RCP = rcppath + '/' + j 
    monthly = [dI for dI in os.listdir(f'{RCP}') if os.path.isdir(os.path.join(f'{RCP}',dI))]
    # 1850-2005
    MON1 = RCP + '/' + monthly[0]
    # 2006-2100
    MON2 = RCP + '/' + monthly[1]
    ### Second.1 loop to get all hist and future data
    for q in range(len(region)):
        # 1850-2005
        CON1 = MON1 + '/' + region[q]
        txtlist1 = []
        for txt1 in os.listdir(f"{CON1}"):
            if txt1.endswith(".txt"):
                txtlist1.append(txt1)
        # 2006-2100
        CON2 = MON2 + '/' + region[q]
        txtlist2 = []
        for txt2 in os.listdir(f"{CON2}"):
            if txt2.endswith(".txt"):
                txtlist2.append(txt2)      
                
                
        #### Second.1.1 loop to read all hist data and caculate the mean
        for k in range(len(txtlist1)):
            if k == 0:
                txtfile = CON1 + '/' + txtlist1[k]
                #print("hist",region[q],txtlist1[k])
                df = pd.read_csv(f"{txtfile}", on_bad_lines='skip')
                try: # some txt file doesnot contain data.
                    df = df[-156:]
                    df.rename(columns= {df.columns[0]:f'Model{k}'}, inplace = True)
                    df = df[f'Model{k}'].str.rsplit("       ",expand=True)
                    df = df[1].str.rsplit("     ", expand=True)
                    df = df[1].str.rsplit("   ", expand=True).add_prefix('nCol_').join(df)
                    df = df[[0,'nCol_0']]
                    df.rename(columns={0: f'Model{k}', 'nCol_0': 'Year'}, inplace=True)
                except:
                    continue
            else:
                txtfile = CON1 + '/' + txtlist1[k]
                #print("hist",region[q],txtlist1[k])
                dt = pd.read_csv(f"{txtfile}", on_bad_lines='skip')
                try: # some txt file doesnot contain data.
                    dt = dt[-156:]
                    dt.rename(columns= {dt.columns[0]:f'Model{k}'}, inplace = True)
                    dt = dt[f'Model{k}'].str.rsplit("       ",expand=True)
                    dt = dt[1].str.rsplit("     ", expand=True)
                    dt = dt[1].str.rsplit("   ", expand=True).add_prefix('nCol_').join(dt)
                    dt = dt[[0,'nCol_0']]
                    dt.rename(columns={0: f'Model{k}', 'nCol_0': 'Year'}, inplace=True)
                except:
                    continue

                df = df.merge(dt, on = 'Year', how = 'left')

        df = df.astype(float)
        df = df.set_index('Year')
        df['mean'] = df.mean(axis=1)
        region1 = [region[q]]*len(df)
        df['Country'] = region1
        df = df[['mean','Country']]
        df.rename(columns= {'mean':f'{j}'},inplace = True)
        dfhist = df 
        
        #### Second.1.2 loop to read all future data and caculate the mean
        df  = pd.DataFrame()
        for k in range(len(txtlist2)):
            if k == 0:
                txtfile = CON2 + '/' + txtlist2[k]
                #print("future",region[q],txtlist2[k])
                df = pd.read_csv(f"{txtfile}", on_bad_lines='skip')
                try: # some txt file doesnot contain data.
                    df = df[-95:]
                    df.rename(columns= {df.columns[0]:f'Model{k}'}, inplace = True)
                    df = df[f'Model{k}'].str.rsplit("       ",expand=True)
                    df = df[1].str.rsplit("     ", expand=True)
                    df = df[1].str.rsplit("   ", expand=True).add_prefix('nCol_').join(df)
                    df = df[[0,'nCol_0']]
                    df.rename(columns={0: f'Model{k}', 'nCol_0': 'Year'}, inplace=True)
                except:
                    continue
            else:
                txtfile = CON2 + '/' + txtlist2[k]
                #print("future",region[q],txtlist2[k])
                dt = pd.read_csv(f"{txtfile}", on_bad_lines='skip')
                try: # some txt file doesnot contain data.
                    dt = dt[-95:]
                    dt.rename(columns= {dt.columns[0]:f'Model{k}'}, inplace = True)
                    dt = dt[f'Model{k}'].str.rsplit("       ",expand=True)
                    dt = dt[1].str.rsplit("     ", expand=True)
                    dt = dt[1].str.rsplit("   ", expand=True).add_prefix('nCol_').join(dt)
                    dt = dt[[0,'nCol_0']]
                    dt.rename(columns={0: f'Model{k}', 'nCol_0': 'Year'}, inplace=True)
                except:
                    continue

                df = df.merge(dt, on = 'Year', how = 'left')

        df = df.astype(float)
        df = df.set_index('Year')
        df['mean'] = df.mean(axis=1)
        region2 = [region[q]]*len(df)
        df['Country'] = region2
        df = df[['mean','Country']]
        df.rename(columns= {'mean':f'{j}'},inplace = True)
        dffuture = df
        
        ## Combine hist and future
        # Create new folders under current directory: region/rcp26,rcp45,rcp60,rcp85
        dd = pd.concat([dfhist, dffuture])
        dd.to_excel(f'region/{j}/{region[q]}{j}.xlsx')

In [10]:
# The txt file does not contain data.
item = pd.read_csv('C:/Users/panda/Downloads/Region/atlas/series/CMIP5one/rcp45/monthly_dump1/world/time_tas_Amon_GFDL-ESM2G_rcp45_r1i1p1_world_mon1_ave12_dump1.txt', on_bad_lines='skip')
item

,# correlate time_tas_Amon_GFDL-ESM2G_rcp45_r1i1p1_world.dat time mon 1 ave 12 begin 2006 standardunits dump time_tas_Amon_GFDL-ESM2G_rcp45_r1i1p1_world_mon1_ave12_dump1.txt
0,# tas [Celsius]
1,# file :: atlas/series/CMIP5/rcp45/monthly/wor...
2,# olderfile :: CMIP5/monthly/tas/tas_Amon_GFDL...
3,# cdi :: Climate Data Interface version 1.5.9 ...
4,# parent_experiment_rip :: r1i1p1
5,# cdo :: Climate Data Operators version 1.5.9 ...
6,# institute_id :: NOAA GFDL
7,# contact :: gfdl.climate.model.info@noaa.gov
8,# project_id :: CMIP5
9,# table_id :: Table Amon (31 Jan 2011)


## Third Part

In [15]:
dire = 'C:/Users/panda/OneDrive - University of Denver/Pardee Center/17 2022 Summer/region'
# dire needs to change
subfolders = [ f.path for f in os.scandir(f"{dire}") if f.is_dir() ]
for p in subfolders:
    excellist = []
    for excel in os.listdir(f"{p}"):
        if excel.endswith(".xlsx"):
            excellist.append(excel)
        for a in range(len(excellist)):
            value = excellist[a][:-5]
            value = value[-5:]
            if a == 0:
                df = pd.read_excel(f'{p}/{excellist[a]}')
            else:
                dt = pd.read_excel(f'{p}/{excellist[a]}')

            final = pd.concat([df,dt])
            final = final.pivot_table(index = 'Country', columns = 'Year', values = f'{value}')
        
        final.to_excel(f"{p}/{value}_regionlevel.xlsx")